In [1]:
# Library
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy
import pickle
import sklearn.ensemble as ek
from sklearn.feature_selection import SelectFromModel
import joblib
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score
import numpy as np
from tqdm import tqdm
import threading
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.svm import SVR
from sklearn.feature_selection import SequentialFeatureSelector
import time

In [2]:
# Input Dataset 

# Define the directory where the parquet files are stored
parquet_directory = "CTU/"

# List all parquet files in the directory
parquet_files = [f for f in os.listdir(parquet_directory) if f.endswith('.parquet')]

# Read each parquet file and append it to a list of DataFrames
dataframes = [pd.read_parquet(os.path.join(parquet_directory, file)) for file in parquet_files]

# Concatenate all DataFrames into a single DataFrame
DM = pd.concat(dataframes, ignore_index=True)

In [3]:
DM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10598771 entries, 0 to 10598770
Data columns (total 11 columns):
 #   Column     Dtype  
---  ------     -----  
 0   dur        float32
 1   proto      object 
 2   dir        object 
 3   state      object 
 4   stos       float32
 5   dtos       float32
 6   tot_pkts   int32  
 7   tot_bytes  int64  
 8   src_bytes  int64  
 9   label      object 
 10  Family     object 
dtypes: float32(3), int32(1), int64(2), object(5)
memory usage: 727.8+ MB


In [4]:
#Feature Selection
features_to_drop = ['proto', 'dir', 'state', 'dtos', 'stos', 'tot_pkts', 'tot_bytes', 'label','Family']

# Droping specified columns and target variable
X = DM.drop(features_to_drop, axis=1).values    
y = DM['label'].values

In [5]:
# Membuat data yang multiclass menjadi binary class
attack_classes = [
    'flow=From-Botnet-V50-1-TCP-Established-SSL-To-Microsoft-1', 
    'flow=From-Botnet-V50-4-TCP-HTTP-Not-Encrypted-Down-2',
    'flow=From-Botnet-V42-UDP-DNS',
    'flow=From-Botnet-V42-TCP-HTTP-Google-Net-Established-6',
    'flow=From-Botnet-V42-TCP-Established',
    'flow=From-Botnet-V50-6-TCP-HTTP-Google-Net-Established-2'
]

# Mmebuat Label Binary
DM['binary_label'] = DM['label'].apply(lambda x: 1 if x in attack_classes else 0)

# Menampilkan rows 1 untuk mengkonformasi label baru
print(DM[['label', 'binary_label']].head())

# Menghitung label Binary
binary_counts = DM['binary_label'].value_counts()
print("Counts of binary labels:")
print(binary_counts)

# Menampilkan sample
print("Sample attack labels:")
print(DM[DM['binary_label'] == 1].head(10))  # Print 10 samples of attack labels
print("Sample benign labels:")
print(DM[DM['binary_label'] == 0].head(10))  # Print 10 samples of benign labels

                             label  binary_label
0      flow=Background-TCP-Attempt             0
1      flow=Background-TCP-Attempt             0
2  flow=Background-TCP-Established             0
3      flow=Background-TCP-Attempt             0
4      flow=Background-TCP-Attempt             0
Counts of binary labels:
binary_label
0    10573337
1       25434
Name: count, dtype: int64
Sample attack labels:
                 dur proto    dir     state  stos  dtos  tot_pkts  tot_bytes  \
1194283     0.000278   udp    <->       CON   0.0   0.0         2        203   
1194500     0.020525   udp    <->       CON   0.0   0.0         2        590   
1194505     0.045125   tcp     ->  SRPA_SPA   0.0   0.0         7        882   
1203011     0.336250   udp    <->       CON   0.0   0.0         2        215   
1203030  3514.083496   tcp     ->   SPA_SPA   0.0   0.0       120       7767   
1204099     0.459301   udp    <->       CON   0.0   0.0         2        212   
1204239     1.371909   udp    <-

In [6]:
#Remove Nan
X = pd.DataFrame(X).dropna()
y = y[X.index]

In [7]:
# Applying Min-Max scaling to make X non-negative
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
# Encode the target variable (if still contains string values)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
# Sequential Feature Selector
sfs_selector = SequentialFeatureSelector(estimator=SVR(kernel="linear"), n_features_to_select=1)
X_sfs = sfs_selector.fit_transform(X_scaled, y_encoded)

In [ ]:
# Get top features selected by SFS
sfs_top_features = np.where(sfs_selector.get_support())[0]

# Get names of selected features
filtered_columns = DM.drop(features_to_drop, axis=1).columns
features = filtered_columns[sfs_top_features]

print("Selected features using Sequential Feature Selector:")
for feature in features:
    print(feature)

In [ ]:
# Tentukan 0.1% dari total dataset
total_samples = int(len(DM) * 0.001)

# Ambil jumlah minimal sampel dari kelas yang lebih sedikit
min_class_samples = DM['binary_label'].value_counts().min()

# Pastikan tidak mengambil lebih dari jumlah minimum kelas yang ada
if total_samples / 2 > min_class_samples:
    total_samples = min_class_samples * 2  # Sesuaikan total sampel agar tidak lebih dari yang tersedia

# Ambil sampel secara acak dari kedua kelas dengan jumlah yang seimbang
sampled_data = DM.groupby('binary_label').apply(lambda x: x.sample(n=int(total_samples / 2), random_state=42)).reset_index(drop=True)

# Cek distribusi kelas setelah sampling
print(sampled_data['binary_label'].value_counts())

hasil_threshold = []

In [ ]:
def Thresholding(threshold, feature):
    # Mengukur waktu mulai
    start_time = time.time()

    # Melakukan thresholding
    BenignDataThreshold = sampled_data[sampled_data[feature] > int(threshold)]
    MalwareDataThreshold = sampled_data[sampled_data[feature] <= int(threshold)]

    # Untuk BenignDataThreshold
    y_true_benign = [0] * len(BenignDataThreshold)
    y_pred_benign = BenignDataThreshold["binary_label"]

    # Untuk MalwareDataThreshold
    y_true_malware = [1] * len(MalwareDataThreshold)
    y_pred_malware = MalwareDataThreshold["binary_label"]

    # Menyatukan true labels dan predicted labels
    y_true = y_true_benign + y_true_malware
    y_pred = y_pred_benign.tolist() + y_pred_malware.tolist()

    # Menghitung confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    # Menghitung Precision, Recall, F1-score, dan Accuracy
    PrecisionThreshold = tp / (tp + fp) if (tp + fp) != 0 else 0
    RecallThreshold = tp / (tp + fn) if (tp + fn) != 0 else 0
    F1ScoreThreshold = 2 * ((PrecisionThreshold * RecallThreshold) / (PrecisionThreshold + RecallThreshold)) if (PrecisionThreshold + RecallThreshold) != 0 else 0
    AccuracyThreshold = (tp + tn) / (tp + fp + tn + fn) if (tp + fp + tn + fn) != 0 else 0

    # Mengukur waktu selesai
    elapsed_time = time.time() - start_time

    # Menyimpan hasil dalam format satu baris untuk setiap threshold
    hasil_threshold.append([feature, threshold, PrecisionThreshold, RecallThreshold, F1ScoreThreshold, AccuracyThreshold, elapsed_time])

# Lakukan thresholding pada fitur yang dipilih oleh Chi-Square Test
for feature in tqdm(features, desc='Progress'):
    # Pastikan bahwa Min dan Max adalah nilai numerik
    Max = pd.to_numeric(sampled_data[feature], errors='coerce').max()
    Min = pd.to_numeric(sampled_data[feature], errors='coerce').min()

    # Cek apakah Min dan Max valid (bukan NaN)
    if pd.notna(Min) and pd.notna(Max):
        for x in range(int(Min), int(Max + 1)):
            Thresholding(x, feature)
    else:
        print(f"Fitur {feature} mengandung nilai non-numerik dan tidak dapat diproses.")

print(hasil_threshold)

# Mengubah hasil menjadi DataFrame dengan kolom yang sesuai
hasil_threshold_df = pd.DataFrame(hasil_threshold, columns=['Feature', 'Threshold', 'Precision', 'Recall', 'F1_Score', 'Accuracy', 'Elapsed_Time'])

# Simpan DataFrame ke CSV
hasil_threshold_df.to_csv('Sequential Feature Selector.csv', index=False)

print("Hasil threshold disimpan ke dalam file 'Sequential Feature Selector.csv'")

In [ ]:
HasilThreshold = pd.read_csv("Sequential Feature Selector.csv")
HasilThreshold.info()

In [ ]:
# Inisialisasi variabel winner untuk masing-masing metrik
winner_precision = ['', '', 0]  # [feature, threshold, value]
winner_recall = ['', '', 0]
winner_f1_score = ['', '', 0]
winner_accuracy = ['', '', 0]
winner_run_time = ['', '', float('inf')]  # Memilih yang terendah, jadi diinisialisasi dengan tak hingga

# Iterasi melalui setiap baris di HasilThreshold
for index, result in HasilThreshold.iterrows():
    # result['Feature'], result['Threshold'], result['Precision'], 
    # result['Recall'], result['F1_Score'], result['Accuracy'],
    # result['Elapsed_Time']
    
    # Membandingkan Precision
    if result['Precision'] > winner_precision[2]:  # result['Precision']
        winner_precision = [result['Feature'], result['Threshold'], result['Precision']]  # [feature, threshold, Precision]
    
    # Membandingkan Recall
    if result['Recall'] > winner_recall[2]:  # result['Recall']
        winner_recall = [result['Feature'], result['Threshold'], result['Recall']]  # [feature, threshold, Recall]

    # Membandingkan F1-score
    if result['F1_Score'] > winner_f1_score[2]:  # result['F1_Score']
        winner_f1_score = [result['Feature'], result['Threshold'], result['F1_Score']]  # [feature, threshold, F1 Score]

    # Membandingkan Accuracy
    if result['Accuracy'] > winner_accuracy[2]:  # result['Accuracy']
        winner_accuracy = [result['Feature'], result['Threshold'], result['Accuracy']]  # [feature, threshold, Accuracy]

    # Membandingkan waktu running (run_time) untuk threshold tercepat (terkecil)
    if result['Elapsed_Time'] < winner_run_time[2]:  # result['Elapsed_Time']
        winner_run_time = [result['Feature'], result['Threshold'], result['Elapsed_Time']]  # [feature, threshold, elapsed_time]

In [ ]:
def plot_metrics_for_winner(winner, metric_name):
    winner_feature, winner_threshold = winner[0], winner[1]

    # Memfilter hasil threshold untuk fitur pemenang
    winner_results = HasilThreshold[HasilThreshold['Feature'] == winner_feature]

    # Pisahkan hasil menjadi list berdasarkan threshold dan kolom metrik
    thresholds = winner_results['Threshold'].tolist()  # Threshold
    precision = winner_results['Precision'].tolist()   # Precision
    recall = winner_results['Recall'].tolist()         # Recall
    f1_score = winner_results['F1_Score'].tolist()     # F1 Score
    accuracy = winner_results['Accuracy'].tolist()      # Accuracy

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(thresholds, accuracy, label='Accuracy', marker='o')
    plt.plot(thresholds, precision, label='Precision', marker='o')
    plt.plot(thresholds, recall, label='Recall', marker='o')
    plt.plot(thresholds, f1_score, label='F1 Score', marker='o')

    plt.title(f'{metric_name} Winner: Metrics vs. Threshold for Feature {winner_feature}')
    plt.xlabel('Threshold')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
def plot_runtime_for_winner(winner):
    winner_feature, winner_threshold = winner[0], winner[1]

    # Memfilter hasil threshold untuk fitur pemenang
    winner_results = HasilThreshold[HasilThreshold['Feature'] == winner_feature]

    # Pisahkan hasil menjadi list berdasarkan threshold dan run_time
    thresholds = winner_results['Threshold'].tolist()    # Threshold
    run_time = winner_results['Elapsed_Time'].tolist()   # Run Time (elapsed_time)

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(thresholds, run_time, label='Run Time', marker='o')

    plt.title(f'Run Time Winner: Run Time vs. Threshold for Feature {winner_feature}')
    plt.xlabel('Threshold')
    plt.ylabel('Run Time (seconds)')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# Plot untuk Accuracy Winner
plot_metrics_for_winner(winner_accuracy, 'Accuracy')

In [ ]:
# Plot untuk Precision Winner
plot_metrics_for_winner(winner_precision, 'Precision')

In [ ]:
# Plot untuk Recall Winner
plot_metrics_for_winner(winner_recall, 'Recall')

In [ ]:
# Plot untuk F1-Score Winner
plot_metrics_for_winner(winner_f1_score, 'F1 Score')

In [ ]:
# Plot untuk Run Time Winner
plot_runtime_for_winner(winner_run_time)

In [ ]:
# Output untuk setiap metrik
print(f"Precision terbaik dicapai dengan fitur {winner_precision[0]} dan threshold {winner_precision[1]}, menghasilkan precision {winner_precision[2]}.")
print(f"Recall terbaik dicapai dengan fitur {winner_recall[0]} dan threshold {winner_recall[1]}, menghasilkan recall {winner_recall[2]}.")
print(f"F1-score terbaik dicapai dengan fitur {winner_f1_score[0]} dan threshold {winner_f1_score[1]}, menghasilkan F1-score {winner_f1_score[2]}.")
print(f"Akurasi terbaik dicapai dengan fitur {winner_accuracy[0]} dan threshold {winner_accuracy[1]}, menghasilkan akurasi {winner_accuracy[2]}.")
print(f"Waktu running tercepat dicapai dengan fitur {winner_run_time[0]} dan threshold {winner_run_time[1]}, dengan waktu running {winner_run_time[2]} detik.")